# Laplacian versus

## Google Colab Integration

In [9]:
# !git clone https://github.com/Stage-SuperResolution/NAME-REPO.git
# !mv NAME-REPO/* .
# !rm -rf NAME-REPO
# !pip install git+https://github.com/akhaten/lasp.git@COMPLETE-COMMIT

## Import

In [10]:
# Lasp module

import sys
sys.path.append('../../..')

import lasp.io
import lasp.filters.linear
import lasp.noise
import lasp.convert
import lasp.algorithm.experimental
import lasp.thresholding
import lasp.differential
import lasp.utils

# Other

import scipy.signal
import scipy.io.matlab
import numpy
import pandas
import tqdm
import pathlib
import matplotlib.pyplot
import typing

IMAGE_PATH = pathlib.Path('../0-Images')
# PATH = pathlib.Path('./3-LaplacianVersus')
# if not(PATH.exists()):
#     PATH.mkdir()

## Test On

### Laplacian with laplacian filter

In [11]:
def mumford_shah_deconv_v2(
    y: numpy.ndarray, 
    h: numpy.ndarray, 
    alpha: float,
    beta: float,
    sigma: float,
    nb_iterations: int,
    tolerance: float,
    error_history: list[float] = None
) -> numpy.ndarray:

    """Mumford Shah
    # TODO: make test
    Solve argmin_{x} { (alpha/2) || y - Hx ||^2 + (beta/2) || nabla y ||^2 + || nabla y ||_1

    Difference with v1 ?
    We use Derivation in fourier space
    """

    kernel2d_id = numpy.pad(numpy.array([[1]]), pad_width=1)

    dx_oper = lasp.differential.dx(kernel2d_id)
    dx_diag = lasp.utils.fourier_diagonalization(dx_oper, y.shape)
    dxT_diag = numpy.conj(dx_diag)

    dy_oper = lasp.differential.dy(kernel2d_id)
    dy_diag = lasp.utils.fourier_diagonalization(dy_oper, y.shape)
    dyT_diag = numpy.conj(dy_diag)

    # Build kernel

    laplacian = lasp.filters.linear.laplacian()
    lap_diag = lasp.utils.fourier_diagonalization(
        kernel = laplacian,
        shape_out = y.shape 
    )
    # lap_diag = Dxt * Dx + Dyt * Dy
    #lap_diag = numpy.abs(Dx)**2 + numpy.abs(Dy)**2
   
    h_diag = lasp.utils.fourier_diagonalization(
        kernel = h,
        shape_out = y.shape
    )

    h2_diag = numpy.abs(h_diag)**2


    uker = alpha * h2_diag + (beta+sigma) * lap_diag

    rhs1fft = alpha * numpy.conj(h_diag) * numpy.fft.fft2(y)

    # Initialization
    u = numpy.copy(y) 
    d_x=numpy.zeros_like(y)
    d_y=numpy.zeros_like(y)
    b_x=numpy.zeros_like(y)
    b_y=numpy.zeros_like(y)

    for _ in range(0, nb_iterations):

        rhs2fft = sigma*dxT_diag*numpy.fft.fft2(d_x-b_x) \
            + sigma*dyT_diag*numpy.fft.fft2(d_y-b_y)
        rhsfft = rhs1fft + rhs2fft

        u_prev = numpy.copy(u)

        u_fft = rhsfft / uker
        u = numpy.real(numpy.fft.ifft2(u_fft))    

        err = numpy.linalg.norm(u-u_prev, 'fro') / numpy.linalg.norm(u, 'fro')
        
        if not(error_history is None):
            error_history.append(err)

        if err < tolerance:
            break
        
    
        u_dx = numpy.real(numpy.fft.ifft2(dx_diag * u_fft))
        u_dy = numpy.real(numpy.fft.ifft2(dy_diag * u_fft))

        d_x, d_y = lasp.thresholding.multidimensional_soft(
            d = numpy.array(
                [ 
                    u_dx + b_x,
                    u_dy + b_y 
                ]
            ),
            epsilon = 1/sigma
        )

        b_x += (u_dx - d_x)
        b_y += (u_dy - d_y)

    u_normalized = lasp.utils.normalize(u)

    return u_normalized

### Laplacian with $\nabla_{x}^{T}\nabla_{x}+\nabla_{y}^{T}\nabla_{y}$

In [12]:
def mumford_shah_deconv_v3(
    y: numpy.ndarray, 
    h: numpy.ndarray, 
    alpha: float,
    beta: float,
    sigma: float,
    nb_iterations: int,
    tolerance: float,
    error_history: list[float] = None
) -> numpy.ndarray:

    """Mumford Shah
    # TODO: make test
    Solve argmin_{x} { (alpha/2) || y - Hx ||^2 + (beta/2) || nabla y ||^2 + || nabla y ||_1

    Difference with v1 ?
    We use Derivation in fourier space
    and we set lap_diag = Dxt * Dx + Dyt * Dy and not laplacia  filter
    """

    kernel2d_id = numpy.pad(numpy.array([[1]]), pad_width=1)

    dx_oper = lasp.differential.dx(kernel2d_id)
    dx_diag = lasp.utils.fourier_diagonalization(dx_oper, y.shape)
    dxT_diag = numpy.conj(dx_diag)

    dy_oper = lasp.differential.dy(kernel2d_id)
    dy_diag = lasp.utils.fourier_diagonalization(dy_oper, y.shape)
    dyT_diag = numpy.conj(dy_diag)

    # Build kernel
    lap_diag = dxT_diag * dx_diag + dyT_diag * dy_diag
   
   
    h_diag = lasp.utils.fourier_diagonalization(
        kernel = h,
        shape_out = y.shape
    )

    h2_diag = numpy.abs(h_diag)**2


    uker = alpha * h2_diag + (beta+sigma) * lap_diag

    rhs1fft = alpha * numpy.conj(h_diag) * numpy.fft.fft2(y)

    # Initialization
    u = numpy.copy(y) 
    d_x=numpy.zeros_like(y)
    d_y=numpy.zeros_like(y)
    b_x=numpy.zeros_like(y)
    b_y=numpy.zeros_like(y)

    for _ in range(0, nb_iterations):

        rhs2fft = sigma*dxT_diag*numpy.fft.fft2(d_x-b_x) \
            + sigma*dyT_diag*numpy.fft.fft2(d_y-b_y)
        rhsfft = rhs1fft + rhs2fft

        u_prev = numpy.copy(u)

        u_fft = rhsfft / uker
        u = numpy.real(numpy.fft.ifft2(u_fft))    

        err = numpy.linalg.norm(u-u_prev, 'fro') / numpy.linalg.norm(u, 'fro')
        
        if not(error_history is None):
            error_history.append(err)

        if err < tolerance:
            break
        
        u_dx = numpy.real(numpy.fft.ifft2(dx_diag * u_fft))
        u_dy = numpy.real(numpy.fft.ifft2(dy_diag * u_fft))

        d_x, d_y = lasp.thresholding.multidimensional_soft(
            d = numpy.array(
                [ 
                    u_dx + b_x, 
                    u_dy + b_y 
                ]
            ),
            epsilon = 1/sigma
        )

        b_x += (u_dx - d_x)
        b_y += (u_dy - d_y)

    u_normalized = lasp.utils.normalize(u)

    return u_normalized

## Dataset utils

In [13]:
def image_from(params: pandas.Series) -> numpy.ndarray:

    image_path = params['image']
    blur = params['blur']
    noise = params['noise']
    
    img = lasp.io.read(image_path)
    out = numpy.copy(img)

    if pandas.notna(blur):
        kernel = lasp.filters.linear.gaussian_filter(size=blur[0], sigma=blur[1])
        out = scipy.signal.convolve2d(out, kernel, mode='same')
    
    if pandas.notna(noise):
        out = lasp.noise.awgn(out, snr=noise)

    return out

def add(
    dataset: pandas.DataFrame,
    image: pathlib.Path,
    deblur_kernel: tuple[int, float],
    alpha: float,
    beta: float,
    sigma: float,
    tol: float = 10**(-4),
    iterations: int = 300,
    noise: float = numpy.nan,
    blur: tuple[int, float] = numpy.nan
) -> pandas.DataFrame:

    to_add = pandas.DataFrame(
        {
            'image' : [image], 
            'deblur_kernel' : [deblur_kernel], 
            'alpha' : [alpha], 
            'beta' : [beta],
            'sigma' : [sigma],
            'tol' : [tol], 
            'iterations' : [iterations], 
            'noise' : [noise],
            'blur' : [blur]
        }
    )

    return pandas.concat([dataset, to_add], ignore_index=True)

def process_from_index(dataset: pandas.DataFrame, index: int) -> tuple[numpy.ndarray, numpy.ndarray]:
   

    params = dataset.loc[index] # Get params from dataset


    out = image_from(params)
    normalized = lasp.utils.normalize(out)

    deblur_kernel = lasp.filters.linear.gaussian_filter(
        size = dataset.iloc[index]['deblur_kernel'][0],
        sigma = dataset.iloc[index]['deblur_kernel'][1]
    )

    # Hyper-parameters
    alpha = params['alpha']
    beta = params['beta']
    sigma = params['sigma']

    # Iterative parameters
    tol = params['tol']
    iterations = params['iterations']

    errors = []

    res = mumford_shah_deconv_v3(
        normalized, deblur_kernel,
        alpha, beta, sigma,
        iterations, tol, 
        errors
    )

    return res, errors

In [14]:
def make_dataset() -> pandas.DataFrame:    
    

    dataset = pandas.DataFrame(
        columns = [
            'image' , 'deblur_kernel',
            'alpha', 'beta', 'sigma', # Hyper-parameters
            'tol', 'iterations', # Iterative parameters
            'noise',
            'blur'
        ]
    )

    dataset = add(
        dataset = dataset, 
        image = IMAGE_PATH / 'Baboon.bmp', deblur_kernel = (7, 3),
        alpha = 100, beta = 1, sigma = 2, 
        tol = 1e-4, iterations = 300, 
        noise = lasp.convert.snrdb_to_snr(30),
        blur = (7, 3)
    )

    return dataset

In [15]:
# DATASET_PATH = PATH / pathlib.Path('dataset.pkl')
dataset = make_dataset()
dataset

,image,deblur_kernel,alpha,beta,sigma,tol,iterations,noise,blur
0,..\0-Images\Baboon.bmp,"(7, 3)",100,1,2,0.0001,300,31.622777,"(7, 3)"


In [16]:
# pandas.to_pickle(dataset, DATASET_PATH)

## Process dataset

In [17]:
begin = 0
# dataset = pandas.read_pickle(DATASET_PATH)
dataset_filtered = dataset.loc[begin:]

res_cmp = []
# errors_cmp = []
for index in tqdm.tqdm(dataset_filtered.index):

    params = dataset.loc[index] # Get params from dataset


    input = image_from(params)
    input_normalized = lasp.utils.normalize(input)

    deblur_kernel = lasp.filters.linear.gaussian_filter(
        size = dataset.iloc[index]['deblur_kernel'][0],
        sigma = dataset.iloc[index]['deblur_kernel'][1]
    )

    # Hyper-parameters
    alpha = params['alpha']
    beta = params['beta']
    sigma = params['sigma']

    # Iterative parameters
    tol = params['tol']
    iterations = params['iterations']

    res_v2 = mumford_shah_deconv_v2(
        input_normalized, deblur_kernel,
        alpha, beta, sigma,
        iterations, tol
    )
    res_v3 = mumford_shah_deconv_v3(
        input_normalized, deblur_kernel,
        alpha, beta, sigma,
        iterations, tol
    )
    
    res_cmp.append(numpy.max(numpy.abs(res_v2-res_v3)))
    # errors_cmp.append(numpy.max(numpy.abs(errors_v1-errors_v2)))

res_cmp = numpy.array(res_cmp)
# errors_cmp = numpy.array(errors_cmp)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:12<00:00, 12.99s/it]


In [18]:
print(res_cmp)

[1.11022302e-15]
